<a href="https://colab.research.google.com/github/noemiefer7/stage_3a/blob/main/GEE_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Utilisation de GEE avec colab pour détecter les zones humides

Importations

In [23]:
import ee
import geemap.core as geemap
import geemap.colormaps as cm

Authentification à GEE et lien vers le projet existant

In [9]:
ee.Authenticate()
ee.Initialize(project='ee-nonoferdinand')



Classification

In [37]:
points =  [[[2.199345427771364, 7.198620909627316],
          [2.199345427771364, 6.2807929477903395],
          [2.704716521521364, 6.2807929477903395],
          [2.704716521521364, 7.198620909627316]]]

polygon = ee.Geometry.Polygon(points)

dataset = ee.ImageCollection('ESA/WorldCover/v100').filter(ee.Filter.bounds(polygon)).first()

visualization = {
    'bands': ['Map'],
}



m = geemap.Map()
m.set_center(2.2747, 6.4128, 10)
m.add_layer(dataset, visualization, 'Landcover')
m.add_layer(polygon)
m

Export.image.toDrive(
    image= dataset,
    description= 'imageToDriveExample',
    scale= 30,
    region= polygon
  )

Map(center=[6.4128, 2.2747], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [15]:
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)


img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .map(mask_edge)
)

desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

spring = ee.Filter.date('2015-03-01', '2015-04-20')
late_spring = ee.Filter.date('2015-04-21', '2015-06-10')
summer = ee.Filter.date('2015-06-11', '2015-08-31')

desc_change = ee.Image.cat(
    desc.filter(spring).mean(),
    desc.filter(late_spring).mean(),
    desc.filter(summer).mean(),
)

asc_change = ee.Image.cat(
    asc.filter(spring).mean(),
    asc.filter(late_spring).mean(),
    asc.filter(summer).mean(),
)



m = geemap.Map()
m.set_center(2.2747, 6.4128, 10)
m.add_layer(asc_change, {'min': -25, 'max': 5}, 'Multi-T Mean ASC', True)
m.add_layer(desc_change, {'min': -25, 'max': 5}, 'Multi-T Mean DESC', True)
m

Map(center=[6.4128, 2.2747], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [20]:
# Load the Sentinel-1 collection, wanted date : 09/29/2022
sentinel_1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2022-09-20','2022-09-30')



# Filter the Sentinel-1 collection by metadata properties.
vv_iw = (
    sentinel_1.filter(
        # Filter to get images with VV and VH dual polarization.
        ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')
    )
    .filter(
        # Filter to get images collected in interferometric wide swath mode.
        ee.Filter.eq('instrumentMode', 'IW')
    )
)

# Separate ascending and descending orbit images into distinct collections.
vv_iw_asc = vv_iw.filter(
    ee.Filter.eq('orbitProperties_pass', 'ASCENDING')
)
vv_iw_desc = vv_iw.filter(
    ee.Filter.eq('orbitProperties_pass', 'DESCENDING')
)

# Calculate temporal means for various observations to use for visualization.
# Mean VH ascending.
vh_iw_asc_mean = vv_iw_asc.select('VV').mean()
# Mean VH descending.
vh_iw_desc_mean = vv_iw_desc.select('VV').mean()
# Mean VV for combined ascending and descending image collections.
vv_iw_asc_desc_mean = vv_iw_asc.merge(vv_iw_desc).select('VV').mean()


# Display the temporal means for various observations, compare them.
m = geemap.Map()
m.addLayer(vv_iw_asc_desc_mean, {'min':-12, 'max':-4}, 'vv_iw')

m.set_center(2.2747, 6.4128, 10)
m


Map(center=[6.4128, 2.2747], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [25]:
dataset = ee.Image('CGIAR/SRTM90_V4')
elevation = dataset.select('elevation')
slope = ee.Terrain.slope(elevation)

palette = cm.palettes.dem

parameters = {'min': 0, 'max': 500, "palette": palette}

m = geemap.Map()
m.set_center(2.2747, 6.4128, 10)
m.add_layer(slope, parameters, 'slope')
#m.add_colorbar({'min': 0, 'max': 60, "palette": palette}, label='Elevation(m)')
m

Map(center=[6.4128, 2.2747], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…